In [ ]:
"""
line_extractor_pt2.py 

This script merges redudant 3D lines based on parallel and proximity conditions.
You can tune the paramters defined in helper.py.

Output:
- Rgb images annotated with extracted lines thier semantic labels. 
- Mesh file(.ply) with all merged 3D lines.
- One 3D line Mesh file(.ply) for each semantic label.
- A numpy file containing all the extracted 2D lines and regressed 3D lines.

Author: Haodong JIANG <221049033@link.cuhk.edu.cn>
Version: 1.0
License: MIT
"""
import numpy as np
import os
import open3d as o3d
# from collections import Counter
from joblib import Parallel, delayed
from scipy import stats
from tqdm import tqdm
import helper

In [ ]:
# load data from pt1
scene_list = ["69e5939669","689fec23D7","c173f62b15","55b2bf8036"]
scene_id = scene_list[2]
### load data path
root_dir = "/data1/home/lucky/ELSED"
scene_data_path = root_dir+f"/SCORE/line_map_extractor/out/{scene_id}/{scene_id}_results_raw.npy"
### result saving path
line_data_folder = root_dir+f"/SCORE/line_map_extractor/out/{scene_id}/" # numpy file with all the extracted 2D lines and merged 3D lines
line_mesh_folder = root_dir+f"/SCORE/line_map_extractor/out/{scene_id}/line_mesh_merged/"
for out_path in [line_data_folder, line_mesh_folder]:
    if not os.path.exists(out_path):
        os.makedirs(out_path)
### load data
scene_data = np.load(scene_data_path, allow_pickle=True).item()
scene_pose = scene_data["scene_pose"]
scene_intrinsic = scene_data["scene_intrinsic"]
label_2_semantic_dict = scene_data["label_2_semantic_dict"]
scene_line_2D_end_points = scene_data["scene_line_2D_end_points"]
scene_line_2D_semantic_labels = scene_data["scene_line_2D_semantic_labels"]
scene_line_2D_params = scene_data["scene_line_2D_params"]
scene_line_2D_match_idx = scene_data["scene_line_2D_match_idx"]
scene_line_3D_end_points = scene_data["scene_line_3D_end_points"]
scene_line_3D_image_source = scene_data["scene_line_3D_image_source"]
scene_line_3D_semantic_labels = scene_data["scene_line_3D_semantic_labels"]

In [ ]:
### preprocessing for 3D line merging
# each 3D line is treated as a vertex on the graph
# the edges are defined by the parallel and proximity conditions
nnode = len(scene_line_3D_semantic_labels)
# precomputing to acclearte the graph construction
pi_list = np.array([(scene_line_3D_end_points[i][0]+scene_line_3D_end_points[i][1]).reshape(1, 3)/2 for i in range(nnode)])
p_diff_list = np.array([(scene_line_3D_end_points[i][1]-scene_line_3D_end_points[i][0]).reshape(1, 3) for i in range(nnode)])
vi_list = np.array([p_diff_list[i]/np.linalg.norm(p_diff_list[i]) for i in range(nnode)])
project_null_list = np.eye(3) - np.einsum('ijk,ijl->ikl', vi_list, vi_list)
print("constructing the consistent graph")
def find_neighbors(i):
    edges_i = []
    edges_j = []
    if i % 1000 == 0:
        print("finding neighbors in progress:", i/nnode*100,"%")
    cur_image_idices = [scene_line_3D_image_source[i]] # cur_image_idices stores the indices of image from which the 3D lines are extracted
    for j in range(i + 1, nnode):
        if scene_line_3D_image_source[j] not in cur_image_idices: # lines extracted from a same image should not be merged
            if abs(np.dot(vi_list[i], vi_list[j].T)) >= helper.params_3D["parrallel_thresh_3D"]: # parallel condition
                if np.linalg.norm(np.dot(project_null_list[i], (pi_list[i] - pi_list[j]).T)) <= helper.params_3D["overlap_thresh_3D"]: # proximity condition
                    edges_i.append(i)
                    edges_j.append(j)
                    cur_image_idices.append(scene_line_3D_image_source[j])
    return edges_i, edges_j
# parallel processing each 3D line
results = Parallel(n_jobs=helper.params_3D["thread_number"])(delayed(find_neighbors)(i) for i in range(nnode))

# # none parallel version
# results = []
# for i in tqdm(range(nnode)):
#     result = find_neighbors(i)
#     results.append(result)

# organize the edges and store as an itermediate result
edges_i = []
edges_j = []
for edges_i_, edges_j_ in results:
    edges_i.extend(edges_i_)
    edges_j.extend(edges_j_)
np.save(line_data_folder+scene_id+"_edges.npy",
        {"edges_i": edges_i, "edges_j": edges_j})

In [ ]:
# load the saved edges
edge_data = np.load(line_data_folder+scene_id+"_edges.npy", allow_pickle=True).item()
edges_i = np.array(edge_data["edges_i"])
edges_j = np.array(edge_data["edges_j"])
nnode = len(scene_line_3D_semantic_labels)
# defines the data structure to be stored
merged_semantic_label_3D =[]
merged_scene_line_3D_end_points=[]

#################### starting merging 3D lines based on graph connectivity ####################
print("# 3D lines before merging",nnode)
mapping = list(range(nnode)) # this list records 3D line merging info
edges_i_ = np.concatenate((edges_i,np.array(range(0,nnode))))
edges_j_ = np.concatenate((edges_j,np.array(range(0,nnode))))
vertex_concat = np.concatenate((edges_i_,edges_j_)) # concatenate the vertices for all th edges

### step 0: remove suspicious lines which are observed only by a small number of images.
unique_elements, counts = np.unique(vertex_concat, return_counts=True)
print(helper.params_3D["degree_threshold"]+2)
vertex_deleted = unique_elements[counts<helper.params_3D["degree_threshold"]+2] # plus 2: count the edge (vi,vi) 
for ver in vertex_deleted:
    mapping[ver]=np.nan
index_deleted = []
for i in range(len(edges_i_)):
    if edges_i_[i] in vertex_deleted or edges_j_[i] in vertex_deleted:
        index_deleted.append(i)
edges_i_=np.delete(edges_i_,index_deleted)
edges_j_=np.delete(edges_j_,index_deleted)

### step 1: iteratively find the vertex with largest degree, merge all its neighbors.  
countt = 0
while len(edges_i_)>0:  
    vertex_concat = np.concatenate((edges_i_,edges_j_))
    mode_result = stats.mode(vertex_concat)
    # if mode_result.count<helper.params_3D["degree_threshold"]+2: 
    ## if the remained lines is not observed by a enough number of images, 
    ## we assume it is a background line and discard it
    #     left_vertex = np.unique(vertex_concat)
    #     for ver in left_vertex:
    #         mapping[ver]=np.nan
    #     break
    most_frequent_index = mode_result.mode
    index_1 = np.where(edges_i_==most_frequent_index)
    index_2 = np.where(edges_j_==most_frequent_index)
    neighbors = np.unique(np.concatenate((edges_j_[index_1], edges_i_[index_2])))  # note that the neighbors contain itself 
    # delete neighbor vertex nodes and remove the edges
    for neighbor in neighbors:
        index_1 = np.where(edges_i_==neighbor)
        index_2 = np.where(edges_j_==neighbor)
        index_delete_neighbor = np.unique(np.concatenate((index_1[0], index_2[0])))
        edges_i_=np.delete(edges_i_,index_delete_neighbor)
        edges_j_=np.delete(edges_j_,index_delete_neighbor)
    # update the end points
    end_points = scene_line_3D_end_points[most_frequent_index]
    v = end_points[1] - end_points[0]
    v = v/np.linalg.norm(v)
    sig_dim = np.argmax(np.abs(v))
    for neighbor in neighbors:
        end_points_temp = scene_line_3D_end_points[neighbor]
        if end_points_temp[0][sig_dim]<end_points[0][sig_dim]:
            end_points[0] = end_points[0] + (end_points_temp[0][sig_dim] - end_points[0][sig_dim])*(v/v[sig_dim]) 
        if end_points_temp[1][sig_dim]>end_points[1][sig_dim]:
            end_points[1] = end_points[1] + (end_points_temp[1][sig_dim] - end_points[1][sig_dim])*(v/v[sig_dim]) 
    # For each unqiue semantic label, we create a 3D line in the map (with same geometric parameters)  
    # Update the mapping list at the same time
    cluster_semantic_labels = []
    for neighbor in neighbors:
        cluster_semantic_labels = np.append(cluster_semantic_labels,scene_line_3D_semantic_labels[neighbor])
    unique_cluster_semantic_lables = np.unique(cluster_semantic_labels)
    unique_cluster_semantic_lables = unique_cluster_semantic_lables[unique_cluster_semantic_lables!=0]
    for label in unique_cluster_semantic_lables:
        merged_semantic_label_3D.append(label)
        merged_scene_line_3D_end_points.append(end_points)
        for neighbor in neighbors:
            if label == scene_line_3D_semantic_labels[neighbor]:
                mapping[neighbor]=len(merged_semantic_label_3D)-1  # from the original index to the new index
                

        
    # Debug: output the 3D line with more than 3 semantic labels
    if len(unique_cluster_semantic_lables)>3: 
        point_diff = end_points[1] - end_points[0]
        point_sets = []
        for sample in range(300):
            point_sets.append(end_points[0] + point_diff*sample/299)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3DVector(point_sets)
        o3d.io.write_point_cloud(line_mesh_folder + f"multiple_semantic_{countt}.ply", pcd)
        countt+=1
        for k in range(0,len(unique_cluster_semantic_lables)):
            print(f"{label_2_semantic_dict[unique_cluster_semantic_lables[k]]},",end="")

#
print("# 3D lines after merging:",len(merged_scene_line_3D_end_points))

### Step 2. Update projection error after merging 3D lines
print("Updating projection error after merging")
scene_projection_error_r = {}
scene_projection_error_t = {}
scene_line_2D_match_idx_updated = {}
for basename in scene_line_2D_match_idx.keys():
    # if basename == 'frame_002180':
    #     print(basename)
    projection_error_r = []
    projection_error_t = []
    intrinsic = scene_intrinsic[basename]
    pose_matrix = np.array(scene_pose[basename])
    line_2D_match_idx = np.array(scene_line_2D_match_idx[basename])
    line_2D_match_idx_updated = line_2D_match_idx.copy()
    for j in range(len(line_2D_match_idx)):
        if np.isnan(line_2D_match_idx[j]) or np.isnan(mapping[line_2D_match_idx[j]]): # no matched line
            # print(line_2D_match_idx[j], mapping[line_2D_match_idx[j]])
            line_2D_match_idx_updated[j] = -1
            projection_error_r.append(np.nan)
            projection_error_t.append(np.nan)
        else:
            mapping_idx = mapping[line_2D_match_idx[j]]
            line_2D_match_idx_updated[j] = mapping_idx
            n_j = scene_line_2D_params[basename][j].reshape(1,3)
            end_points_3D = merged_scene_line_3D_end_points[mapping_idx]
            v = end_points_3D[1] - end_points_3D[0]
            v = v/np.linalg.norm(v)
            # compute the projection error
            error_rot,error_trans = helper.calculate_error(n_j,v,intrinsic,pose_matrix,end_points_3D[0],end_points_3D[1])
            projection_error_r.append(np.abs(error_rot))
            projection_error_t.append(np.abs(error_trans)) 
            if np.abs(error_trans) > 0.1:
                print(f"Warning: projection error too large for {basename} at index {j}, error_t={error_trans}")
    scene_line_2D_match_idx_updated[basename] = line_2D_match_idx_updated          
    scene_projection_error_r[basename] = projection_error_r
    scene_projection_error_t[basename] = projection_error_t

In [ ]:
### save data for relocalization experiements
np.save(line_data_folder+scene_id+"_results_merged.npy", {
    "scene_pose": scene_pose,
    "scene_intrinsic": scene_intrinsic,
    "label_2_semantic_dict": label_2_semantic_dict,
    ###
    "scene_line_2D_semantic_labels": scene_line_2D_semantic_labels,
    "scene_line_2D_params": scene_line_2D_params,
    "scene_line_2D_end_points": scene_line_2D_end_points,
    "scene_line_2D_match_idx_updated": scene_line_2D_match_idx_updated,
    "scene_projection_error_r": scene_projection_error_r,
    "scene_projection_error_t": scene_projection_error_t,
    ###
    "merged_scene_line_3D_semantic_labels": merged_semantic_label_3D,
    "merged_scene_line_3D_end_points": merged_scene_line_3D_end_points,
    ###
    "params_3D": helper.params_3D
})

In [ ]:
### output 3D line mesh for visualization 
# save all merged 3D lines
point_sets=[]
for i in range(len(merged_semantic_label_3D)):
    end_points = merged_scene_line_3D_end_points[i]
    point_diff = end_points[1] - end_points[0]
    for sample in range(300):
        point_sets.append(end_points[0] + point_diff*sample/299)
point_sets = np.vstack(point_sets)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_sets)
o3d.io.write_point_cloud(line_mesh_folder+scene_id+f"_merged_3D_line_mesh.ply", pcd)

# save the 3D line mesh for each semantic label
def process_label(i, semantic_label):
    if int(semantic_label) == 0:
        return
    index = np.where(merged_semantic_label_3D == semantic_label)
    print("semantic label:" + f"{label_2_semantic_dict[int(semantic_label)]}" + " number of lines:", len(index[0]))
    point_sets = []
    for j in range(len(index[0])):
        end_points = merged_scene_line_3D_end_points[i]
        point_diff = end_points[1] - end_points[0]
        for sample in range(300):
            point_sets.append(end_points[0] + point_diff*sample/299)
    if point_sets:
        point_sets = np.vstack(point_sets)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_sets)
        o3d.io.write_point_cloud(line_mesh_folder + f"{label_2_semantic_dict[int(semantic_label)]}.ply", pcd)
semantic_labels_all = np.unique(merged_semantic_label_3D)
# Parallel(n_jobs=params_3D["thread_number"])(delayed(process_label)(i, semantic_label) for i, semantic_label in enumerate(semantic_labels_all))
Parallel(n_jobs=8)(delayed(process_label)(i, semantic_label) for i, semantic_label in enumerate(semantic_labels_all))

